In [1]:
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
connections = pd.read_csv('data/connections.csv', 
                   sep='\t', 
                   header=0)
connections.head(10)

,ts,imei,mwra,c.katana,c.android.gm,c.dogalize,c.android.chrome,c.android.youtube,c.raider,c.android.vending,c.UCMobile.x86,c.updateassist,c.UCMobile.intl
0,2018-05-05 10:00:00,359043379931766452,0.0,11.06115,12.03060,9.91193,11.61488,14.41566,0.00630,50.61159,59.89217,10.61973,29.26605
1,2018-05-05 10:01:00,3590433799317661479,1.0,10.36927,13.33058,10.47183,5.81122,13.42850,0.00335,42.15179,33.30329,37.55184,79.39223
2,2018-05-05 10:02:00,863033069630348941,1.0,11.13179,11.55974,10.19114,9.48536,11.48909,0.00687,31.98880,57.96025,98.36767,67.51165
3,2018-05-05 10:03:00,863033069630348891,0.0,18.55737,7.52229,9.76952,10.09019,8.59293,11.20737,63.24643,68.14477,1.25216,70.80992
4,2018-05-05 10:04:00,863033069630348982,1.0,16.23621,6.46232,14.43755,10.06295,7.56208,1.10024,52.21213,62.91701,39.23750,4.40841
5,2018-05-05 10:05:00,359043379931766890,0.0,12.65449,11.31760,12.80750,7.63353,6.94398,0.03072,46.84296,72.10419,65.21307,53.36385
6,2018-05-05 10:06:00,8630330696303482394,1.0,7.51715,9.71631,11.16790,14.25744,11.86472,0.00040,55.64279,51.34929,11.43310,93.70164
7,2018-05-05 10:07:00,8630330696303481552,1.0,10.42239,11.16408,15.36993,12.40764,15.72517,0.00342,39.94261,78.25351,78.85671,51.24171
8,2018-05-05 10:08:00,8630330696303482410,0.0,16.96537,7.81786,11.74992,8.24555,13.41690,2.28102,51.88202,51.86428,77.53953,71.13616
9,2018-05-05 10:09:00,8630330696303481289,0.0,11.72449,10.31466,7.08285,9.68121,13.50829,0.01218,41.56365,63.83457,63.27459,1.02142


In [3]:
processes = pd.read_csv('data/processes.csv', 
                   sep='\t', 
                   header=0)
processes.head(10)

,ts,imei,mwra,p.android.gm,p.android.chrome,p.android.packageinstaller,p.android.externalstorage,p.android.settings,p.android.documentsui,p.system,...,p.inputmethod.latin,p.katana,p.process.gapps,p.browser.provider,p.gms.persistent,p.android.vending,p.notifier,p.simulator,p.dogalize,p.android.defcontainer
0,2018-05-05 10:00:00,359043379931766452,0.0,4.76403,10.88071,11.51360,11.78927,9.99354,6.87053,13.51346,...,79.29044,10.79383,83.63731,6.32371,98.02549,68.85903,71.53462,2.25715,93.42594,21.75571
1,2018-05-05 10:01:00,3590433799317661479,1.0,11.46891,8.28988,7.09764,13.77786,15.06889,12.69941,12.31459,...,45.98593,99.59445,98.05942,7.98917,95.75324,84.32885,28.67616,33.34162,74.46069,50.49767
2,2018-05-05 10:02:00,863033069630348941,1.0,14.17447,10.60933,10.48707,11.42372,12.21700,15.63323,10.68789,...,80.90579,39.15136,35.96190,39.82428,23.28994,11.66910,73.73339,94.80023,29.08949,53.56893
3,2018-05-05 10:03:00,863033069630348891,0.0,13.24902,12.97128,14.07949,12.04242,15.05862,14.44617,13.38765,...,73.85500,85.74165,89.95390,23.31018,85.76856,83.81530,77.31641,77.72672,82.13086,77.27342
4,2018-05-05 10:04:00,863033069630348982,1.0,10.03368,9.47160,8.07917,14.44551,15.64555,8.80989,7.84002,...,19.27745,26.95881,61.93643,71.64654,85.38242,49.78533,37.98503,90.32310,39.01098,5.83207
5,2018-05-05 10:05:00,359043379931766890,0.0,12.10487,7.06790,10.77449,11.54052,13.25847,8.67721,8.88003,...,63.77575,66.61781,32.08551,48.18337,34.87881,99.56064,23.62307,23.62557,30.64260,16.19268
6,2018-05-05 10:06:00,8630330696303482394,1.0,11.20547,13.54670,6.01785,13.97898,9.35211,8.64164,14.59811,...,62.71154,6.37995,49.26883,14.26143,35.61887,99.91404,78.01157,0.10921,85.78794,3.44839
7,2018-05-05 10:07:00,8630330696303481552,1.0,9.43152,11.19937,10.16669,13.59798,12.60449,14.07398,15.60397,...,68.55050,74.45278,46.79925,99.01885,12.66284,62.62353,22.11113,9.42886,74.13334,84.01403
8,2018-05-05 10:08:00,8630330696303482410,0.0,9.40552,11.05332,10.70500,12.36928,11.72829,12.71607,11.03679,...,80.48845,68.66819,78.93986,70.78193,4.54058,3.58426,84.71937,40.42517,54.45794,30.66241
9,2018-05-05 10:09:00,8630330696303481289,0.0,7.14433,11.25230,14.82949,14.69164,13.77117,11.75249,10.76002,...,28.58429,11.06029,92.74713,92.41559,40.25147,33.18383,59.26034,77.34647,32.94507,63.39220


In [4]:
# Odstranenie duplicit
connections.drop_duplicates( inplace=True)
processes.drop_duplicates(inplace=True)

In [5]:
# Spojenie tabuliek Connections a Processes podla imei a ts
df = pd.merge(connections, processes, on=['imei', 'ts','mwra'], how='outer')
df.drop(columns=["ts"], inplace=True)

In [6]:
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)

    return a[(a > upper) | (a < lower)]

In [7]:
def filter_outliers(df,column,action):
    outliers = identify_outliers(df[column])
    if action == 'delete':
        df.drop(index=outliers.index,inplace=True)
    elif action == 'replace':
        lower_bound = df[column].quantile(0.05)
        upper_bound = df[column].quantile(0.95)

        df[column] = np.where(
            df[column] < lower_bound, lower_bound,
            np.where(df[column] > upper_bound, upper_bound, df[column])
        )

In [8]:
def prepareData(X,Y):
    columns_to_edit = [
        'c.android.gm', 'c.dogalize', 'c.android.chrome', 'c.android.youtube', 
        'c.katana', 'c.android.vending', 'c.UCMobile.x86', 'c.updateassist', 
        'c.UCMobile.intl', 'p.android.gm', 'p.android.chrome', 'p.android.packageinstaller', 
        'p.android.externalstorage', 'p.android.settings', 'p.android.documentsui', 
        'p.system', 'p.google', 'p.android.gms', 'p.olauncher', 'c.katana'
    ]
    
    for column in columns_to_edit:
        filter_outliers(X, column, action='delete')
        
    # Synchronizacia zaznamov x_train s y_train a reset indexov
    Y = Y.loc[X.index]
    X = X.reset_index(drop=True)
    Y = Y.reset_index(drop=True)
    
    return X,Y

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# k = pocet atributov kt. chcem aby selector vybral
def createPipeline(scaling='standard',transformation='quantile',k=10):
    
    # Na c.raider pouzijeme transformaciu
    if transformation == 'quantile':
        preprocessor = ColumnTransformer(
        transformers=[
            ('quantile', QuantileTransformer(output_distribution='normal', random_state=0), ['c.raider']),
        ],
        remainder='passthrough'  # Zvyšné stĺpce necháme zmeny, aby sme ich mohli následne škálovať
    )
    else:
          preprocessor = ColumnTransformer(
        transformers=[
            ('power', PowerTransformer(method='yeo-johnson', standardize=True), ['c.raider']),
        ],
        remainder='passthrough'
    )
        
    if scaling == 'standard':
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', StandardScaler()),
            ('feature_selection', SelectKBest(mutual_info_classif, k=k)),
        ])
    else:
         pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('scaler', RobustScaler()),
            ('feature_selection', SelectKBest(mutual_info_classif, k=k)),
        ])
    
    return pipeline

In [41]:
# Shuffle Samples
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Rozdelenie dat na trenovaciu a testovaciu sadu
X = df.drop(columns='mwra')
Y = df['mwra']
x_Train, x_Test, y_Train, y_Test =  train_test_split(X, Y, test_size=0.2, random_state=1)

In [42]:
# Vytvorenie pipeline
# S atributmi scaling a transformation mozem vytvorit rozne pipeliny
pipeline = createPipeline(k=10,scaling='robust',transformation='yeo-johnson')

In [43]:
x_Train, y_Train = prepareData(x_Train, y_Train)
x_Test, y_Test = prepareData(x_Test, y_Test)

In [44]:
transformed_train_data = pipeline.fit_transform(x_Train, y_Train)
transformed_test_data = pipeline.transform(x_Test)

# Vytiahneme si nazvy vsetkych atributov
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

# Vyberieme len atribúty, ktoré vybral SelectKBest
selected_features = [feature_names[i] for i in range(len(feature_names)) if pipeline.named_steps['feature_selection'].get_support()[i]]
selected_features = [name.split('__')[-1] for name in selected_features]


x_Train = pd.DataFrame(transformed_train_data, columns=selected_features)
x_Train.head()

,c.raider,c.katana,c.android.gm,c.android.chrome,c.android.youtube,p.android.gm,p.android.packageinstaller,p.android.settings,p.android.documentsui,p.system
0,-0.146710,-0.577623,-0.116354,0.656540,0.448679,-0.743180,0.101828,-1.416351,-1.382635,0.282384
1,-0.151733,-0.666478,-0.012955,-0.573187,0.702290,-0.180799,0.238011,0.719907,0.568905,0.293647
2,-0.143366,-0.533340,0.249637,0.805767,0.274549,0.323796,0.014010,-0.464098,0.091842,-0.461114
3,0.207114,0.231311,0.058304,0.468629,0.801099,-0.760389,0.702899,-0.693218,-0.985072,0.391380
4,-0.153409,-0.678069,-0.567658,1.705517,1.273791,-0.487436,0.651338,-1.265769,-1.748530,-0.213121


In [45]:
x_Test = pd.DataFrame(transformed_test_data, columns=selected_features)
x_Test.head()

,c.raider,c.katana,c.android.gm,c.android.chrome,c.android.youtube,p.android.gm,p.android.packageinstaller,p.android.settings,p.android.documentsui,p.system
0,-0.160963,-1.044200,1.355644,0.456577,0.973852,-0.694650,1.154020,-0.408725,1.327414,1.086564
1,0.036511,0.058327,-0.634317,0.542482,0.248593,-0.846346,0.415402,-0.922493,-0.084838,-1.051479
2,2.124472,0.863752,-0.074542,-0.262908,-0.081956,-0.004651,0.172747,-0.077873,0.312589,-0.456994
3,0.218637,0.241984,-0.244052,0.005160,-0.557946,0.339849,1.025244,0.005661,-0.056295,0.118944
4,0.596984,0.449589,1.251447,-0.249325,0.562052,-0.854064,-0.630340,-0.175867,0.404437,1.028221


# 3.Faza Strojové učenie

## 3.1 Jednoduchý klasifikátor na základe závislosti v dátach  (5b)
### (A-3b) Naimplementujte jednoduchý ID3 klasifikátor s hĺbkou min 2 (vrátane root/koreň). 

In [68]:
id3_classifier = DecisionTreeClassifier(criterion='entropy', max_depth=7)

id3_classifier.fit(x_Train, y_Train)

y_pred = id3_classifier.predict(x_Test)

### (B-1b) Vyhodnoťte Váš ID3 klasifikátor pomocou metrík accuracy, precision a recall.

In [69]:
accuracy = accuracy_score(y_Test, y_pred)
precision = precision_score(y_Test, y_pred, average='weighted')
recall = recall_score(y_Test, y_pred, average='weighted')   

print("Accuracy", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy 0.8796433878157504
Precision: 0.8791940154892164
Recall: 0.8796433878157504


### (C-1b) Zístite či Váš ID3 klasifikátor má overfit.

In [71]:
# Zistime metriky na trenovacich datach
y_train_pred = id3_classifier.predict(x_Train)
train_accuracy = accuracy_score(y_Train, y_train_pred)
train_precision = precision_score(y_Train, y_train_pred, average='weighted')
train_recall = recall_score(y_Train, y_train_pred, average='weighted')

print("Train Accuracy", train_accuracy)
print("Train Precision:", train_precision)
print("Train Recall:", train_recall)

Train Accuracy 0.9033202231775359
Train Precision: 0.902997606146135
Train Recall: 0.9033202231775359


### ID3 klasifikátor nemá overfit pretože rozdiel medzi výsledkami na trenovácich dátach a testovacich je minimalny

## 3.2 Trénovanie a vyhodnotenie klasifikátorov strojového učenia (5b)
### (A-1b) Na trénovanie využite jeden stromový algoritmus v scikit-learn.

### (B-1b) Porovnajte s jedným iným nestromovým algoritmom v scikit-learn.

### (C-1b) Porovnajte výsledky s ID3 z prvého kroku.

### (D-1b) Vizualizujte natrénované pravidlá minimálne pre jeden Vami vybraný algoritmus

### (E-1b) Vyhodnoťte natrénované modely pomocou metrík accuracy, precision a recall

## 3.3 Optimalizácia alias hyperparameter tuning (5b)
### (A-1b) Vyskúšajte rôzne nastavenie hyperparametrov (tuning) pre zvolený algoritmus tak, aby ste optimalizovali výkonnosť (bez underfitingu)

### (B-1b) Vyskúšajte kombinácie modelov (ensemble) pre zvolený algoritmus tak, aby ste optimalizovali výkonnosť (bez underfitingu)

### (C-1b) Využite krížovú validáciu (cross validation) na trénovacej množine.

### (D-2b) Dokážte že Váš nastavený najlepší model je bez overfitingu.

## 3.4 Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasifikáciu (5b) 
### (A-1b) Stratégie riešenia chýbajúcich hodnôt a outlierov

### (B-1b) Dátová transformácia (scaling, transformer, …)

### (C-1b) Výber atribútov, výber algoritmov, hyperparameter tuning, ensemble learning

### (D-1b) Ktorý model je Váš najlepší model pre nasadenie (deployment)? 

### (E-1b) Aký je data pipeline pre jeho vybudovanie na základe Vášho datasetu v produkcii?